<a href="https://colab.research.google.com/github/kodenshacho/sigma/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn as nn
from torch.nn import functional as F
import math

from basicsr.utils.registry import ARCH_REGISTRY
from .arch_util import default_init_weights, make_layer, pixel_unshuffle


class ResidualDenseBlock(nn.Module):
    def __init__(self, num_feat=64, num_grow_ch=32):
        super().__init__()
        self.conv1 = nn.Conv2d(num_feat, num_grow_ch, 3, 1, 1)
        self.conv2 = nn.Conv2d(num_feat + num_grow_ch, num_grow_ch, 3, 1, 1)
        self.conv3 = nn.Conv2d(num_feat + 2 * num_grow_ch, num_grow_ch, 3, 1, 1)
        self.conv4 = nn.Conv2d(num_feat + 3 * num_grow_ch, num_grow_ch, 3, 1, 1)
        self.conv5 = nn.Conv2d(num_feat + 4 * num_grow_ch, num_feat, 3, 1, 1)
        self.lrelu = nn.LeakyReLU(0.2, inplace=True)
        default_init_weights([self.conv1, self.conv2, self.conv3, self.conv4, self.conv5], 0.1)

    def forward(self, x):
        x1 = self.lrelu(self.conv1(x))
        x2 = self.lrelu(self.conv2(torch.cat((x, x1), 1)))
        x3 = self.lrelu(self.conv3(torch.cat((x, x1, x2), 1)))
        x4 = self.lrelu(self.conv4(torch.cat((x, x1, x2, x3), 1)))
        x5 = self.conv5(torch.cat((x, x1, x2, x3, x4), 1))
        return x5 * 0.2 + x


class RRDB(nn.Module):
    def __init__(self, num_feat, num_grow_ch=32):
        super().__init__()
        self.rdb1 = ResidualDenseBlock(num_feat, num_grow_ch)
        self.rdb2 = ResidualDenseBlock(num_feat, num_grow_ch)
        self.rdb3 = ResidualDenseBlock(num_feat, num_grow_ch)

    def forward(self, x):
        out = self.rdb1(x)
        out = self.rdb2(out)
        out = self.rdb3(out)
        return out * 0.2 + x


@ARCH_REGISTRY.register()
class RRDBNet(nn.Module):
    def __init__(self, num_in_ch, num_out_ch, scale=1, num_feat=64, num_block=23, num_grow_ch=32):
        super().__init__()
        self.scale = scale
        if scale == 2:
            num_in_ch *= 4
        elif scale == 1:
            num_in_ch *= 16

        self.conv_first = nn.Conv2d(num_in_ch, num_feat, 3, 1, 1)
        self.body = make_layer(RRDB, num_block, num_feat=num_feat, num_grow_ch=num_grow_ch)
        self.conv_body = nn.Conv2d(num_feat, num_feat, 3, 1, 1)

        # FC layer at smallest node
        self.fc_smallest = nn.Linear(num_feat, num_feat, bias=False)
        self.init_fc_identity()

        # upsampling layers
        self.conv_up1 = nn.Conv2d(num_feat, num_feat, 3, 1, 1)
        self.conv_up2 = nn.Conv2d(num_feat, num_feat, 3, 1, 1)
        self.conv_hr = nn.Conv2d(num_feat, num_feat, 3, 1, 1)
        self.conv_last = nn.Conv2d(num_feat, num_out_ch, 3, 1, 1)
        self.lrelu = nn.LeakyReLU(0.2, inplace=True)

    def forward(self, x):
        if self.scale == 2:
            feat = pixel_unshuffle(x, scale=2)
        elif self.scale == 1:
            feat = pixel_unshuffle(x, scale=4)
        else:
            feat = x
        feat = self.conv_first(feat)
        body_feat = self.conv_body(self.body(feat))
        feat = feat + body_feat

        # Store feat before FC for comparison
        feat_before_fc = feat.clone()

        # apply FC without altering spatial dims
        b, c, h, w = feat.shape
        feat_flat = feat.permute(0,2,3,1).reshape(-1, c)
        with torch.no_grad():
            feat_fc = F.linear(feat_flat, self.fc_smallest.weight)
        feat = feat_fc.reshape(b, h, w, c).permute(0,3,1,2)


        # apply FC without altering spatial dims
        # b, c, h, w = feat.shape
        # feat_flat = feat.permute(0,2,3,1).reshape(-1, c)
        # feat_fc = self.fc_smallest(feat_flat)
        # feat = feat_fc.reshape(b, h, w, c).permute(0,3,1,2)

        # Compare feat before and after FC
        print('Feat before FC shape:', feat_before_fc.shape)
        print('Feat after FC shape:', feat.shape)
        print('Feat max abs diff before and after FC:', torch.max(torch.abs(feat_before_fc - feat)).item())
        # Add more detailed comparison
        print('Feat mean abs diff before and after FC:', torch.mean(torch.abs(feat_before_fc - feat)).item())
        print('Are feats close? (allclose):', torch.allclose(feat_before_fc, feat))


        feat = self.lrelu(self.conv_up1(F.interpolate(feat, scale_factor=2, mode='nearest')))
        feat = self.lrelu(self.conv_up2(F.interpolate(feat, scale_factor=2, mode='nearest')))
        out = self.conv_last(self.lrelu(self.conv_hr(feat)))
        return out

    def init_fc_identity(self):
        # Identity 초기화 후 학습되지 않도록 weight 고정
        with torch.no_grad():
            eye = torch.eye(self.fc_smallest.weight.size(0), device=self.fc_smallest.weight.device)
            self.fc_smallest.weight.copy_(eye)
        self.fc_smallest.weight.requires_grad_(False)

    def init_fc_random(self):
        # 랜덤 초기화 (optional, weight 고정 해제하지 않음)
        nn.init.kaiming_uniform_(self.fc_smallest.weight, a=math.sqrt(5))

    def print_fc_weights(self, prefix='FC Weights'):
        print(f'{prefix}:')
        print(self.fc_smallest.weight.data)

# 해당 수정으로 identity 가중치가 고정되므로 반복 Iteration마다 동일 출력을 보장합니다.


In [ ]:

# 테스트 스크립트 예시
if __name__ == '__main__':
    model = RRDBNet(3, 3, scale=1)
    pretrained = RRDBNet(3, 3, scale=1)
    # Create a dummy pretrained model file for demonstration
    dummy_pretrained_path = 'pretrained_1600x1200.pth'
    if not os.path.exists(dummy_pretrained_path):
        print(f"Creating dummy pretrained model file at {dummy_pretrained_path}")
        torch.save(pretrained.state_dict(), dummy_pretrained_path)

    pretrained.load_state_dict(torch.load(dummy_pretrained_path))

    # 사전학습 모델 FC 가중치 출력
    pretrained.print_fc_weights(prefix='Pretrained FC Weights')

    # 훈련 모델에 사전학습 가중치 복사
    model.load_state_dict(pretrained.state_dict())
    # 훈련 모델 FC 가중치 출력
    model.print_fc_weights(prefix='Loaded Model FC Weights')

    # 차이 비교
    diff = pretrained.fc_smallest.weight.data - model.fc_smallest.weight.data
    print('FC Weight Max abs diff:', diff.abs().max().item())

    # 동일한 입력 비교
    x = torch.randn(1,3,1600,1200)
    out_pre = pretrained(x)
    out_model = model(x)
    print('Output Max abs diff:', torch.max(torch.abs(out_pre - out_model)).item())

In [ ]:
# https://github.com/boomb0om/Real-ESRGAN-colab
!git clone https://github.com/XPixelGroup/BasicSR.git


Cloning into 'BasicSR'...
remote: Enumerating objects: 5924, done.
remote: Total 5924 (delta 0), reused 0 (delta 0), pack-reused 5924 (from 1)
Receiving objects: 100% (5924/5924), 4.14 MiB | 20.77 MiB/s, done.
Resolving deltas: 100% (3758/3758), done.


In [ ]:
# Googleドライブのマウントと作業フォルダへの移動
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/work'

In [ ]:
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# upload with fc rrdbnet...

In [ ]:
# パッケージのインストール
!pip install basicsr facexlib gfpgan
!pip install -r requirements.txt
!python setup.py develop


In [ ]:
! python tests/test_model.py
# ! pip install --upgrade torchvision

# Task
Analyze the Real-ESRGAN model architecture in "/content/Real-ESRGAN/BasicSR/basicsr/archs/rrdbnet_arch.py" to identify the convolution layer with the smallest number of nodes. Modify the architecture by inserting a fully connected layer after this identified convolution layer, ensuring the fully connected layer's input size matches the preceding layer's output size. Prepare the model for training with 1600x1200 input and output, aiming to achieve results similar to the original pre-trained Real-ESRGAN model.

## 모델 아키텍처 분석

### Subtask:
Real-ESRGAN 모델의 아키텍처, 특히 `RRDBNet`의 구조를 분석하여 fully connected layer를 삽입할 위치를 결정합니다. 노드 수가 가장 작은 콘볼루션 레이어의 출력 크기를 확인합니다.


**Reasoning**:
The subtask requires analyzing the `RRDBNet` architecture defined in a Python file to identify the convolution layer with the smallest number of output nodes and its output size. This involves reading the file content and examining the class definition, which can be done by reading the file and printing its content.



In [ ]:
file_path = '/content/Real-ESRGAN/basicsr/archs/rrdbnet_arch.py'
try:
    with open(file_path, 'r') as f:
        file_content = f.read()
    print(file_content)
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Error: File not found at /content/Real-ESRGAN/basicsr/archs/rrdbnet_arch.py


**Reasoning**:
The previous command failed because the file path was incorrect. The `BasicSR` repository was cloned, and the `Real-ESRGAN` repository was also cloned and the working directory was changed to `Real-ESRGAN`. The `rrdbnet_arch.py` file is located within the `basicsr` directory inside the cloned `BasicSR` repository, not within the `Real-ESRGAN` directory. The correct path should be within the cloned `BasicSR` directory. I need to adjust the file path and try reading the file again.



In [ ]:
file_path = '/content/BasicSR/basicsr/archs/rrdbnet_arch.py'
try:
    with open(file_path, 'r') as f:
        file_content = f.read()
    print(file_content)
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Error: File not found at /content/BasicSR/basicsr/archs/rrdbnet_arch.py


**Reasoning**:
The previous command also failed because the file path is still incorrect. The `BasicSR` repository was cloned into `/content/BasicSR`. The file should be located inside the `BasicSR` directory. I will check the directory structure and try reading the file again with the correct path.



In [ ]:
import os
print(os.listdir('/content/BasicSR/basicsr/archs'))

FileNotFoundError: [Errno 2] No such file or directory: '/content/BasicSR/basicsr/archs'

**Reasoning**:
The error indicates that the directory `/content/BasicSR/basicsr/archs` does not exist. This suggests that the cloning of the `BasicSR` repository might not have placed the files in the expected location or the path is still incorrect. I will list the contents of the `/content` directory and then the `/content/BasicSR` directory to understand the file structure after cloning.



In [ ]:
import os
print(os.listdir('/content'))
print(os.listdir('/content/BasicSR'))

['.config', 'Real-ESRGAN', 'sample_data']


FileNotFoundError: [Errno 2] No such file or directory: '/content/BasicSR'

**Reasoning**:
The error indicates that the `/content/BasicSR` directory does not exist, even though the previous output showed that the `BasicSR` repository was cloned. This is unexpected. It seems the cloning step might have failed or the output from the first cell was misleading. Given the goal is to analyze the `RRDBNet` architecture which is part of the `BasicSR` library, and `BasicSR` was supposedly installed in a later step (`!pip install basicsr`), the source code for `RRDBNet` should be available within the Python site-packages. I will try to locate the `rrdbnet_arch.py` file within the site-packages directory and read its content.



In [ ]:
import site
import os

site_packages = site.getsitepackages()
file_path = None
for site_package in site_packages:
    possible_path = os.path.join(site_package, 'basicsr', 'archs', 'rrdbnet_arch.py')
    if os.path.exists(possible_path):
        file_path = possible_path
        break

if file_path:
    print(f"Found rrdbnet_arch.py at: {file_path}")
    try:
        with open(file_path, 'r') as f:
            file_content = f.read()
        print(file_content)
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
else:
    print("Could not find rrdbnet_arch.py in site-packages.")


Found rrdbnet_arch.py at: /usr/local/lib/python3.11/dist-packages/basicsr/archs/rrdbnet_arch.py
import torch
from torch import nn as nn
from torch.nn import functional as F

from basicsr.utils.registry import ARCH_REGISTRY
from .arch_util import default_init_weights, make_layer, pixel_unshuffle


class ResidualDenseBlock(nn.Module):
    """Residual Dense Block.

    Used in RRDB block in ESRGAN.

    Args:
        num_feat (int): Channel number of intermediate features.
        num_grow_ch (int): Channels for each growth.
    """

    def __init__(self, num_feat=64, num_grow_ch=32):
        super(ResidualDenseBlock, self).__init__()
        self.conv1 = nn.Conv2d(num_feat, num_grow_ch, 3, 1, 1)
        self.conv2 = nn.Conv2d(num_feat + num_grow_ch, num_grow_ch, 3, 1, 1)
        self.conv3 = nn.Conv2d(num_feat + 2 * num_grow_ch, num_grow_ch, 3, 1, 1)
        self.conv4 = nn.Conv2d(num_feat + 3 * num_grow_ch, num_grow_ch, 3, 1, 1)
        self.conv5 = nn.Conv2d(num_feat + 4 * num_grow_ch

## 모델 아키텍처 수정

### Subtask:
분석 결과를 바탕으로 `RRDBNet` 아키텍처에 fully connected layer를 삽입하는 코드를 작성합니다. fully connected layer의 입력 크기는 이전 레이어의 출력 크기와 일치하도록 설정합니다.


**Reasoning**:
To modify the `RRDBNet` architecture, I need to read the original `rrdbnet_arch.py` file, insert the fully connected layer and the logic to handle it, and then write the modified code back to the file. I will add the fully connected layer after the `conv1` layer inside the `ResidualDenseBlock` as it has the smallest number of nodes (`num_grow_ch`).



In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

from basicsr.utils.registry import ARCH_REGISTRY


class ResidualDenseBlock(nn.Module):
    """Residual Dense Block.

    Used in RRDB block in ESRGAN.

    Args:
        num_feat (int): Channel number of intermediate features.
        num_grow_ch (int): Channels for growth.
    """

    def __init__(self, num_feat=64, num_grow_ch=32):
        super(ResidualDenseBlock, self).__init__()
        self.conv1 = nn.Conv2d(num_feat, num_grow_ch, 3, 1, 1)
        self.conv2 = nn.Conv2d(num_feat + num_grow_ch, num_grow_ch, 3, 1, 1)
        self.conv3 = nn.Conv2d(num_feat + 2 * num_grow_ch, num_grow_ch, 3, 1, 1)
        self.conv4 = nn.Conv2d(num_feat + 3 * num_grow_ch, num_grow_ch, 3, 1, 1)
        self.conv5 = nn.Conv2d(num_feat + 4 * num_grow_ch, num_feat, 3, 1, 1)

        self.lrelu = nn.LakyReLU(negative_slope=0.2, inplace=True)

        # New fully connected layer
        # The input size is num_grow_ch * H * W. We will determine H and W in the forward pass.
        # For now, we initialize it without a specific size, and will re-initialize it in the forward pass
        # This is a common practice when the input size is dynamic.
        self.fc = None


    def forward(self, x):
        """Forward function.

        Args:
            x (Tensor): Input tensor with shape (n, c, h, w).

        Returns:
            Tensor: Forward results.
        """
        x1 = self.lrelu(self.conv1(x))

        # Flatten the output of conv1 and pass it through the fully connected layer
        n, c, h, w = x1.size()
        if self.fc is None:
            self.fc = nn.Linear(c * h * w, c * h * w).to(x.device)

        x1_flat = x1.view(n, -1)
        x1_fc = self.fc(x1_flat)
        x1 = x1_fc.view(n, c, h, w)


        x2 = self.lrelu(self.conv2(torch.cat((x, x1), 1)))
        x3 = self.lrelu(self.conv3(torch.cat((x, x1, x2), 1)))
        x4 = self.lrelu(self.conv4(torch.cat((x, x1, x2, x3), 1)))
        x5 = self.conv5(torch.cat((x, x1, x2, x3, x4), 1))
        # Empirically, we find that scaling the residual block by a small
        # constant improves training stability.
        return x5 * 0.2 + x


class RRDB(nn.Module):
    """Residual in Residual Dense Block.

    Args:
        num_feat (int): Channel number of intermediate features.
        num_grow_ch (int): Channels for growth.
    """

    def __init__(self, num_feat, num_grow_ch=32):
        super(RRDB, self).__init__()
        self.rdb1 = ResidualDenseBlock(num_feat, num_grow_ch)
        self.rdb2 = ResidualDenseBlock(num_feat, num_grow_ch)
        self.rdb3 = ResidualDenseBlock(num_feat, num_grow_ch)

    def forward(self, x):
        """Forward function.

        Args:
            x (Tensor): Input tensor with shape (n, c, h, w).

        Returns:
            Tensor: Forward results.
        """
        out = self.rdb1(x)
        out = self.rdb2(out)
        out = self.rdb3(out)
        # Empirically, we find that scaling the residual block by a small
        # constant improves training stability.
        return out * 0.2 + x


@ARCH_REGISTRY.register()
class RRDBNet(nn.Module):
    """Networks consisting of Residual in Residual Dense Block, which is used
    in ESRGAN.

    ESRGAN: Enhanced Super-Resolution Generative Adversarial Networks.

    Args:
        num_in_ch (int): Channel number of inputs.
        num_out_ch (int): Channel number of outputs.
        num_feat (int): Channel number of intermediate features.
            Default: 64
        num_block (int): Block number in the trunk network. Default: 23
        num_grow_ch (int): Channels for growth. Default: 32
    """

    def __init__(self, num_in_ch, num_out_ch, num_feat=64, num_block=23, num_grow_ch=32):
        super(RRDBNet, self).__init__()
        self.conv_first = nn.Conv2d(num_in_ch, num_feat, 3, 1, 1)
        self.body = nn.Sequential(*[RRDB(num_feat, num_grow_ch) for _ in range(num_block)])
        self.conv_body = nn.Conv2d(num_feat, num_feat, 3, 1, 1)
        # upsampling
        self.conv_up1 = nn.Conv2d(num_feat, num_feat, 3, 1, 1)
        self.conv_up2 = nn.Conv2d(num_feat, num_feat, 3, 1, 1)
        self.conv_hr = nn.Conv2d(num_feat, num_feat, 3, 1, 1)
        self.conv_last = nn.Conv2d(num_feat, num_out_ch, 3, 1, 1)

        self.lrelu = nn.LakyReLU(negative_slope=0.2, inplace=True)

    def forward(self, x):
        """Forward function.

        Args:
            x (Tensor): Input tensor with shape (n, c, h, w).

        Returns:
            Tensor: Forward results.
        """
        feat = self.conv_first(x)
        body_feat = self.conv_body(self.body(feat))
        feat = feat + body_feat
        # upsampling
        feat = self.lrelu(self.conv_up1(F.interpolate(feat, scale_factor=2, mode='nearest')))
        feat = self.lrelu(self.conv_up2(F.interpolate(feat, scale_factor=2, mode='nearest')))
        out = self.conv_last(self.lrelu(self.conv_hr(feat)))
        return out


file_path = '/usr/local/lib/python3.11/dist-packages/basicsr/archs/rrdbnet_arch.py'

with open(file_path, 'w') as f:
    # We need to write the imports and the classes to the file.
    # We will reconstruct the file content with the modified ResidualDenseBlock.

    # Imports
    f.write("import torch\n")
    f.write("from torch import nn\n")
    f.write("from torch.nn import functional as F\n\n")
    f.write("from basicsr.utils.registry import ARCH_REGISTRY\n\n\n")

    # ResidualDenseBlock
    f.write("class ResidualDenseBlock(nn.Module):\n")
    f.write("    \"\"\"Residual Dense Block.\n\n")
    f.write("    Used in RRDB block in ESRGAN.\n\n")
    f.write("    Args:\n")
    f.write("        num_feat (int): Channel number of intermediate features.\n")
    f.write("        num_grow_ch (int): Channels for growth.\n")
    f.write("    \"\"\"\n\n")
    f.write("    def __init__(self, num_feat=64, num_grow_ch=32):\n")
    f.write("        super(ResidualDenseBlock, self).__init__()\n")
    f.write("        self.conv1 = nn.Conv2d(num_feat, num_grow_ch, 3, 1, 1)\n")
    f.write("        self.conv2 = nn.Conv2d(num_feat + num_grow_ch, num_grow_ch, 3, 1, 1)\n")
    f.write("        self.conv3 = nn.Conv2d(num_feat + 2 * num_grow_ch, num_grow_ch, 3, 1, 1)\n")
    f.write("        self.conv4 = nn.Conv2d(num_feat + 3 * num_grow_ch, num_grow_ch, 3, 1, 1)\n")
    f.write("        self.conv5 = nn.Conv2d(num_feat + 4 * num_grow_ch, num_feat, 3, 1, 1)\n\n")
    f.write("        self.lrelu = nn.LakyReLU(negative_slope=0.2, inplace=True)\n\n")
    f.write("        self.fc = None\n\n")
    f.write("    def forward(self, x):\n")
    f.write("        \"\"\"Forward function.\n\n")
    f.write("        Args:\n")
    f.write("            x (Tensor): Input tensor with shape (n, c, h, w).\n\n")
    f.write("        Returns:\n")
    f.write("            Tensor: Forward results.\n")
    f.write("        \"\"\"\n")
    f.write("        x1 = self.lrelu(self.conv1(x))\n\n")
    f.write("        n, c, h, w = x1.size()\n")
    f.write("        if self.fc is None:\n")
    f.write("            self.fc = nn.Linear(c * h * w, c * h * w).to(x.device)\n\n")
    f.write("        x1_flat = x1.view(n, -1)\n")
    f.write("        x1_fc = self.fc(x1_flat)\n")
    f.write("        x1 = x1_fc.view(n, c, h, w)\n\n")
    f.write("        x2 = self.lrelu(self.conv2(torch.cat((x, x1), 1)))\n")
    f.write("        x3 = self.lrelu(self.conv3(torch.cat((x, x1, x2), 1)))\n")
    f.write("        x4 = self.lrelu(self.conv4(torch.cat((x, x1, x2, x3), 1)))\n")
    f.write("        x5 = self.conv5(torch.cat((x, x1, x2, x3, x4), 1))\n")
    f.write("        return x5 * 0.2 + x\n\n\n")

    # RRDB
    f.write("class RRDB(nn.Module):\n")
    f.write("    \"\"\"Residual in Residual Dense Block.\n\n")
    f.write("    Args:\n")
    f.write("        num_feat (int): Channel number of intermediate features.\n")
    f.write("        num_grow_ch (int): Channels for growth.\n")
    f.write("    \"\"\"\n\n")
    f.write("    def __init__(self, num_feat, num_grow_ch=32):\n")
    f.write("        super(RRDB, self).__init__()\n")
    f.write("        self.rdb1 = ResidualDenseBlock(num_feat, num_grow_ch)\n")
    f.write("        self.rdb2 = ResidualDenseBlock(num_feat, num_grow_ch)\n")
    f.write("        self.rdb3 = ResidualDenseBlock(num_feat, num_grow_ch)\n\n")
    f.write("    def forward(self, x):\n")
    f.write("        \"\"\"Forward function.\n\n")
    f.write("        Args:\n")
    f.write("            x (Tensor): Input tensor with shape (n, c, h, w).\n\n")
    f.write("        Returns:\n")
    f.write("            Tensor: Forward results.\n")
    f.write("        \"\"\"\n")
    f.write("        out = self.rdb1(x)\n")
    f.write("        out = self.rdb2(out)\n")
    f.write("        out = self.rdb3(out)\n")
    f.write("        return out * 0.2 + x\n\n\n")

    # RRDBNet
    f.write("@ARCH_REGISTRY.register()\n")
    f.write("class RRDBNet(nn.Module):\n")
    f.write("    \"\"\"Networks consisting of Residual in Residual Dense Block, which is used\n")
    f.write("    in ESRGAN.\n\n")
    f.write("    ESRGAN: Enhanced Super-Resolution Generative Adversarial Networks.\n\n")
    f.write("    Args:\n")
    f.write("        num_in_ch (int): Channel number of inputs.\n")
    f.write("        num_out_ch (int): Channel number of outputs.\n")
    f.write("        num_feat (int): Channel number of intermediate features.\n")
    f.write("            Default: 64\n")
    f.write("        num_block (int): Block number in the trunk network. Default: 23\n")
    f.write("        num_grow_ch (int): Channels for growth. Default: 32\n")
    f.write("    \"\"\"\n\n")
    f.write("    def __init__(self, num_in_ch, num_out_ch, num_feat=64, num_block=23, num_grow_ch=32):\n")
    f.write("        super(RRDBNet, self).__init__()\n")
    f.write("        self.conv_first = nn.Conv2d(num_in_ch, num_feat, 3, 1, 1)\n")
    f.write("        self.body = nn.Sequential(*[RRDB(num_feat, num_grow_ch) for _ in range(num_block)])\n")
    f.write("        self.conv_body = nn.Conv2d(num_feat, num_feat, 3, 1, 1)\n")
    f.write("        # upsampling\n")
    f.write("        self.conv_up1 = nn.Conv2d(num_feat, num_feat, 3, 1, 1)\n")
    f.write("        self.conv_up2 = nn.Conv2d(num_feat, num_feat, 3, 1, 1)\n")
    f.write("        self.conv_hr = nn.Conv2d(num_feat, num_feat, 3, 1, 1)\n")
    f.write("        self.conv_last = nn.Conv2d(num_feat, num_out_ch, 3, 1, 1)\n\n")
    f.write("        self.lrelu = nn.LakyReLU(negative_slope=0.2, inplace=True)\n\n")
    f.write("    def forward(self, x):\n")
    f.write("        \"\"\"Forward function.\n\n")
    f.write("        Args:\n")
    f.write("            x (Tensor): Input tensor with shape (n, c, h, w).\n\n")
    f.write("        Returns:\n")
    f.write("            Tensor: Forward results.\n")
    f.write("        \"\"\"\n")
    f.write("        feat = self.conv_first(x)\n")
    f.write("        body_feat = self.conv_body(self.body(feat))\n")
    f.write("        feat = feat + body_feat\n")
    f.write("        # upsampling\n")
    f.write("        feat = self.lrelu(self.conv_up1(F.interpolate(feat, scale_factor=2, mode='nearest')))\n")
    f.write("        feat = self.lrelu(self.conv_up2(F.interpolate(feat, scale_factor=2, mode='nearest')))\n")
    f.write("        out = self.conv_last(self.lrelu(self.conv_hr(feat)))\n")
    f.write("        return out\n")

print(f"Successfully modified and saved {file_path}")

ModuleNotFoundError: No module named 'torchvision.transforms.functional_tensor'

**Reasoning**:
The previous code block failed because the `basicsr` library, which was installed, has a dependency issue with `torchvision`. The error `ModuleNotFoundError: No module named 'torchvision.transforms.functional_tensor'` indicates that the installed version of `torchvision` is not compatible with the `basicsr` code that was executed during the import. To fix this, I will try upgrading `torchvision` to a version that is likely compatible. After upgrading, I will attempt to modify the `rrdbnet_arch.py` file again.



In [ ]:
! pip install --upgrade torchvision

import torch
from torch import nn
from torch.nn import functional as F

from basicsr.utils.registry import ARCH_REGISTRY


class ResidualDenseBlock(nn.Module):
    """Residual Dense Block.

    Used in RRDB block in ESRGAN.

    Args:
        num_feat (int): Channel number of intermediate features.
        num_grow_ch (int): Channels for growth.
    """

    def __init__(self, num_feat=64, num_grow_ch=32):
        super(ResidualDenseBlock, self).__init__()
        self.conv1 = nn.Conv2d(num_feat, num_grow_ch, 3, 1, 1)
        self.conv2 = nn.Conv2d(num_feat + num_grow_ch, num_grow_ch, 3, 1, 1)
        self.conv3 = nn.Conv2d(num_feat + 2 * num_grow_ch, num_grow_ch, 3, 1, 1)
        self.conv4 = nn.Conv2d(num_feat + 3 * num_grow_ch, num_grow_ch, 3, 1, 1)
        self.conv5 = nn.Conv2d(num_feat + 4 * num_grow_ch, num_feat, 3, 1, 1)

        self.lrelu = nn.LakyReLU(negative_slope=0.2, inplace=True)

        # New fully connected layer
        # The input size is num_grow_ch * H * W. We will determine H and W in the forward pass.
        # For now, we initialize it without a specific size, and will re-initialize it in the forward pass
        # This is a common practice when the input size is dynamic.
        self.fc = None


    def forward(self, x):
        """Forward function.

        Args:
            x (Tensor): Input tensor with shape (n, c, h, w).

        Returns:
            Tensor: Forward results.
        """
        x1 = self.lrelu(self.conv1(x))

        # Flatten the output of conv1 and pass it through the fully connected layer
        n, c, h, w = x1.size()
        if self.fc is None:
            self.fc = nn.Linear(c * h * w, c * h * w).to(x.device)

        x1_flat = x1.view(n, -1)
        x1_fc = self.fc(x1_flat)
        x1 = x1_fc.view(n, c, h, w)


        x2 = self.lrelu(self.conv2(torch.cat((x, x1), 1)))
        x3 = self.lrelu(self.conv3(torch.cat((x, x1, x2), 1)))
        x4 = self.lrelu(self.conv4(torch.cat((x, x1, x2, x3), 1)))
        x5 = self.conv5(torch.cat((x, x1, x2, x3, x4), 1))
        # Empirically, we find that scaling the residual block by a small
        # constant improves training stability.
        return x5 * 0.2 + x


class RRDB(nn.Module):
    """Residual in Residual Dense Block.

    Args:
        num_feat (int): Channel number of intermediate features.
        num_grow_ch (int): Channels for growth.
    """

    def __init__(self, num_feat, num_grow_ch=32):
        super(RRDB, self).__init__()
        self.rdb1 = ResidualDenseBlock(num_feat, num_grow_ch)
        self.rdb2 = ResidualDenseBlock(num_feat, num_grow_ch)
        self.rdb3 = ResidualDenseBlock(num_feat, num_grow_ch)

    def forward(self, x):
        """Forward function.

        Args:
            x (Tensor): Input tensor with shape (n, c, h, w).

        Returns:
            Tensor: Forward results.
        """
        out = self.rdb1(x)
        out = self.rdb2(out)
        out = self.rdb3(out)
        # Empirically, we find that scaling the residual block by a small
        # constant improves training stability.
        return out * 0.2 + x


@ARCH_REGISTRY.register()
class RRDBNet(nn.Module):
    """Networks consisting of Residual in Residual Dense Block, which is used
    in ESRGAN.

    ESRGAN: Enhanced Super-Resolution Generative Adversarial Networks.

    Args:
        num_in_ch (int): Channel number of inputs.
        num_out_ch (int): Channel number of outputs.
        num_feat (int): Channel number of intermediate features.
            Default: 64
        num_block (int): Block number in the trunk network. Default: 23
        num_grow_ch (int): Channels for growth. Default: 32
    """

    def __init__(self, num_in_ch, num_out_ch, num_feat=64, num_block=23, num_grow_ch=32):
        super(RRDBNet, self).__init__()
        self.conv_first = nn.Conv2d(num_in_ch, num_feat, 3, 1, 1)
        self.body = nn.Sequential(*[RRDB(num_feat, num_grow_ch) for _ in range(num_block)])
        self.conv_body = nn.Conv2d(num_feat, num_feat, 3, 1, 1)
        # upsampling
        self.conv_up1 = nn.Conv2d(num_feat, num_feat, 3, 1, 1)
        self.conv_up2 = nn.Conv2d(num_feat, num_feat, 3, 1, 1)
        self.conv_hr = nn.Conv2d(num_feat, num_feat, 3, 1, 1)
        self.conv_last = nn.Conv2d(num_feat, num_out_ch, 3, 1, 1)

        self.lrelu = nn.LakyReLU(negative_slope=0.2, inplace=True)

    def forward(self, x):
        """Forward function.

        Args:
            x (Tensor): Input tensor with shape (n, c, h, w).

        Returns:
            Tensor: Forward results.
        """
        feat = self.conv_first(x)
        body_feat = self.conv_body(self.body(feat))
        feat = feat + body_feat
        # upsampling
        feat = self.lrelu(self.conv_up1(F.interpolate(feat, scale_factor=2, mode='nearest')))
        feat = self.lrelu(self.conv_up2(F.interpolate(feat, scale_factor=2, mode='nearest')))
        out = self.conv_last(self.lrelu(self.conv_hr(feat)))
        return out


file_path = '/usr/local/lib/python3.11/dist-packages/basicsr/archs/rrdbnet_arch.py'

with open(file_path, 'w') as f:
    # We need to write the imports and the classes to the file.
    # We will reconstruct the file content with the modified ResidualDenseBlock.

    # Imports
    f.write("import torch\n")
    f.write("from torch import nn\n")
    f.write("from torch.nn import functional as F\n\n")
    f.write("from basicsr.utils.registry import ARCH_REGISTRY\n\n\n")

    # ResidualDenseBlock
    f.write("class ResidualDenseBlock(nn.Module):\n")
    f.write("    \"\"\"Residual Dense Block.\n\n")
    f.write("    Used in RRDB block in ESRGAN.\n\n")
    f.write("    Args:\n")
    f.write("        num_feat (int): Channel number of intermediate features.\n")
    f.write("        num_grow_ch (int): Channels for growth.\n")
    f.write("    \"\"\"\n\n")
    f.write("    def __init__(self, num_feat=64, num_grow_ch=32):\n")
    f.write("        super(ResidualDenseBlock, self).__init__()\n")
    f.write("        self.conv1 = nn.Conv2d(num_feat, num_grow_ch, 3, 1, 1)\n")
    f.write("        self.conv2 = nn.Conv2d(num_feat + num_grow_ch, num_grow_ch, 3, 1, 1)\n")
    f.write("        self.conv3 = nn.Conv2d(num_feat + 2 * num_grow_ch, num_grow_ch, 3, 1, 1)\n")
    f.write("        self.conv4 = nn.Conv2d(num_feat + 3 * num_grow_ch, num_grow_ch, 3, 1, 1)\n")
    f.write("        self.conv5 = nn.Conv2d(num_feat + 4 * num_grow_ch, num_feat, 3, 1, 1)\n\n")
    f.write("        self.lrelu = nn.LakyReLU(negative_slope=0.2, inplace=True)\n\n")
    f.write("        self.fc = None\n\n")
    f.write("    def forward(self, x):\n")
    f.write("        \"\"\"Forward function.\n\n")
    f.write("        Args:\n")
    f.write("            x (Tensor): Input tensor with shape (n, c, h, w).\n\n")
    f.write("        Returns:\n")
    f.write("            Tensor: Forward results.\n")
    f.write("        \"\"\"\n")
    f.write("        x1 = self.lrelu(self.conv1(x))\n\n")
    f.write("        n, c, h, w = x1.size()\n")
    f.write("        if self.fc is None: # Initialize FC layer dynamically based on input size\n")
    f.write("            self.fc = nn.Linear(c * h * w, c * h * w).to(x.device)\n\n")
    f.write("        x1_flat = x1.view(n, -1)\n")
    f.write("        x1_fc = self.fc(x1_flat)\n")
    f.write("        x1 = x1_fc.view(n, c, h, w)\n\n")
    f.write("        x2 = self.lrelu(self.conv2(torch.cat((x, x1), 1)))\n")
    f.write("        x3 = self.lrelu(self.conv3(torch.cat((x, x1, x2), 1)))\n")
    f.write("        x4 = self.lrelu(self.conv4(torch.cat((x, x1, x2, x3), 1)))\n")
    f.write("        x5 = self.conv5(torch.cat((x, x1, x2, x3, x4), 1))\n")
    f.write("        return x5 * 0.2 + x\n\n\n")

    # RRDB
    f.write("class RRDB(nn.Module):\n")
    f.write("    \"\"\"Residual in Residual Dense Block.\n\n")
    f.write("    Args:\n")
    f.write("        num_feat (int): Channel number of intermediate features.\n")
    f.write("        num_grow_ch (int): Channels for growth.\n")
    f.write("    \"\"\"\n\n")
    f.write("    def __init__(self, num_feat, num_grow_ch=32):\n")
    f.write("        super(RRDB, self).__init__()\n")
    f.write("        self.rdb1 = ResidualDenseBlock(num_feat, num_grow_ch)\n")
    f.write("        self.rdb2 = ResidualDenseBlock(num_feat, num_grow_ch)\n")
    f.write("        self.rdb3 = ResidualDenseBlock(num_feat, num_grow_ch)\n\n")
    f.write("    def forward(self, x):\n")
    f.write("        \"\"\"Forward function.\n\n")
    f.write("        Args:\n")
    f.write("            x (Tensor): Input tensor with shape (n, c, h, w).\n\n")
    f.write("        Returns:\n")
    f.write("            Tensor: Forward results.\n")
    f.write("        \"\"\"\n")
    f.write("        out = self.rdb1(x)\n")
    f.write("        out = self.rdb2(out)\n")
    f.write("        out = self.rdb3(out)\n")
    f.write("        return out * 0.2 + x\n\n\n")

    # RRDBNet
    f.write("@ARCH_REGISTRY.register()\n")
    f.write("class RRDBNet(nn.Module):\n")
    f.write("    \"\"\"Networks consisting of Residual in Residual Dense Block, which is used\n")
    f.write("    in ESRGAN.\n\n")
    f.write("    ESRGAN: Enhanced Super-Resolution Generative Adversarial Networks.\n\n")
    f.write("    Args:\n")
    f.write("        num_in_ch (int): Channel number of inputs.\n")
    f.write("        num_out_ch (int): Channel number of outputs.\n")
    f.write("        num_feat (int): Channel number of intermediate features.\n")
    f.write("            Default: 64\n")
    f.write("        num_block (int): Block number in the trunk network. Default: 23\n")
    f.write("        num_grow_ch (int): Channels for growth. Default: 32\n")
    f.write("    \"\"\"\n\n")
    f.write("    def __init__(self, num_in_ch, num_out_ch, num_feat=64, num_block=23, num_grow_ch=32):\n")
    f.write("        super(RRDBNet, self).__init__()\n")
    f.write("        self.conv_first = nn.Conv2d(num_in_ch, num_feat, 3, 1, 1)\n")
    f.write("        self.body = nn.Sequential(*[RRDB(num_feat, num_grow_ch) for _ in range(num_block)])\n")
    f.write("        self.conv_body = nn.Conv2d(num_feat, num_feat, 3, 1, 1)\n")
    f.write("        # upsampling\n")
    f.write("        self.conv_up1 = nn.Conv2d(num_feat, num_feat, 3, 1, 1)\n")
    f.write("        self.conv_up2 = nn.Conv2d(num_feat, num_feat, 3, 1, 1)\n")
    f.write("        self.conv_hr = nn.Conv2d(num_feat, num_feat, 3, 1, 1)\n")
    f.write("        self.conv_last = nn.Conv2d(num_feat, num_out_ch, 3, 1, 1)\n\n")
    f.write("        self.lrelu = nn.LakyReLU(negative_slope=0.2, inplace=True)\n\n")
    f.write("    def forward(self, x):\n")
    f.write("        \"\"\"Forward function.\n\n")
    f.write("        Args:\n")
    f.write("            x (Tensor): Input tensor with shape (n, c, h, w).\n\n")
    f.write("        Returns:\n")
    f.write("            Tensor: Forward results.\n")
    f.write("        \"\"\"\n")
    f.write("        feat = self.conv_first(x)\n")
    f.write("        body_feat = self.conv_body(self.body(feat))\n")
    f.write("        feat = feat + body_feat\n")
    f.write("        # upsampling\n")
    f.write("        feat = self.lrelu(self.conv_up1(F.interpolate(feat, scale_factor=2, mode='nearest')))\n")
    f.write("        feat = self.lrelu(self.conv_up2(F.interpolate(feat, scale_factor=2, mode='nearest')))\n")
    f.write("        out = self.conv_last(self.lrelu(self.conv_hr(feat)))\n")
    f.write("        return out\n")

print(f"Attempted to upgrade torchvision and rewrite {file_path}")

AssertionError: An object named 'RRDBNet' was already registered in 'arch' registry!

**Reasoning**:
The previous command failed because the `RRDBNet` class was already registered in the `ARCH_REGISTRY`. This is likely because the `basicsr` library was imported when the code block started, and that import triggered the registration of the original `RRDBNet` class. To fix this, I need to avoid the initial import of `basicsr` and directly write the modified code to the file. I will remove the import statements for `basicsr.utils.registry` and the `@ARCH_REGISTRY.register()` decorator from the code that writes to the file, as these are causing the re-registration error. The registration will happen when the modified file is imported later.



In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

# Do not import ARCH_REGISTRY here to avoid re-registration issues

file_path = '/usr/local/lib/python3.11/dist-packages/basicsr/archs/rrdbnet_arch.py'

with open(file_path, 'w') as f:
    # Imports
    f.write("import torch\n")
    f.write("from torch import nn\n")
    f.write("from torch.nn import functional as F\n\n")
    # Write the ARCH_REGISTRY import here, so it is imported when the file is loaded
    f.write("from basicsr.utils.registry import ARCH_REGISTRY\n\n\n")

    # ResidualDenseBlock
    f.write("class ResidualDenseBlock(nn.Module):\n")
    f.write("    \"\"\"Residual Dense Block.\n\n")
    f.write("    Used in RRDB block in ESRGAN.\n\n")
    f.write("    Args:\n")
    f.write("        num_feat (int): Channel number of intermediate features.\n")
    f.write("        num_grow_ch (int): Channels for growth.\n")
    f.write("    \"\"\"\n\n")
    f.write("    def __init__(self, num_feat=64, num_grow_ch=32):\n")
    f.write("        super(ResidualDenseBlock, self).__init__()\n")
    f.write("        self.conv1 = nn.Conv2d(num_feat, num_grow_ch, 3, 1, 1)\n")
    f.write("        self.conv2 = nn.Conv2d(num_feat + num_grow_ch, num_grow_ch, 3, 1, 1)\n")
    f.write("        self.conv3 = nn.Conv2d(num_feat + 2 * num_grow_ch, num_grow_ch, 3, 1, 1)\n")
    f.write("        self.conv4 = nn.Conv2d(num_feat + 3 * num_grow_ch, num_grow_ch, 3, 1, 1)\n")
    f.write("        self.conv5 = nn.Conv2d(num_feat + 4 * num_grow_ch, num_feat, 3, 1, 1)\n\n")
    f.write("        self.lrelu = nn.LakyReLU(negative_slope=0.2, inplace=True)\n\n")
    f.write("        self.fc = None\n\n")
    f.write("    def forward(self, x):\n")
    f.write("        \"\"\"Forward function.\n\n")
    f.write("        Args:\n")
    f.write("            x (Tensor): Input tensor with shape (n, c, h, w).\n\n")
    f.write("        Returns:\n")
    f.write("            Tensor: Forward results.\n")
    f.write("        \"\"\"\n")
    f.write("        x1 = self.lrelu(self.conv1(x))\n\n")
    f.write("        n, c, h, w = x1.size()\n")
    f.write("        if self.fc is None: # Initialize FC layer dynamically based on input size\n")
    f.write("            self.fc = nn.Linear(c * h * w, c * h * w).to(x.device)\n\n")
    f.write("        x1_flat = x1.view(n, -1)\n")
    f.write("        x1_fc = self.fc(x1_flat)\n")
    f.write("        x1 = x1_fc.view(n, c, h, w)\n\n")
    f.write("        x2 = self.lrelu(self.conv2(torch.cat((x, x1), 1)))\n")
    f.write("        x3 = self.lrelu(self.conv3(torch.cat((x, x1, x2), 1)))\n")
    f.write("        x4 = self.lrelu(self.conv4(torch.cat((x, x1, x2, x3), 1)))\n")
    f.write("        x5 = self.conv5(torch.cat((x, x1, x2, x3, x4), 1))\n")
    f.write("        return x5 * 0.2 + x\n\n\n")

    # RRDB
    f.write("class RRDB(nn.Module):\n")
    f.write("    \"\"\"Residual in Residual Dense Block.\n\n")
    f.write("    Args:\n")
    f.write("        num_feat (int): Channel number of intermediate features.\n")
    f.write("        num_grow_ch (int): Channels for growth.\n")
    f.write("    \"\"\"\n\n")
    f.write("    def __init__(self, num_feat, num_grow_ch=32):\n")
    f.write("        super(RRDB, self).__init__()\n")
    f.write("        self.rdb1 = ResidualDenseBlock(num_feat, num_grow_ch)\n")
    f.write("        self.rdb2 = ResidualDenseBlock(num_feat, num_grow_ch)\n")
    f.write("        self.rdb3 = ResidualDenseBlock(num_feat, num_grow_ch)\n\n")
    f.write("    def forward(self, x):\n")
    f.write("        \"\"\"Forward function.\n\n")
    f.write("        Args:\n")
    f.write("            x (Tensor): Input tensor with shape (n, c, h, w).\n\n")
    f.write("        Returns:\n")
    f.write("            Tensor: Forward results.\n")
    f.write("        \"\"\"\n")
    f.write("        out = self.rdb1(x)\n")
    f.write("        out = self.rdb2(out)\n")
    f.write("        out = self.rdb3(out)\n")
    f.write("        return out * 0.2 + x\n\n\n")

    # RRDBNet
    # Add the ARCH_REGISTRY decorator here
    f.write("@ARCH_REGISTRY.register()\n")
    f.write("class RRDBNet(nn.Module):\n")
    f.write("    \"\"\"Networks consisting of Residual in Residual Dense Block, which is used\n")
    f.write("    in ESRGAN.\n\n")
    f.write("    ESRGAN: Enhanced Super-Resolution Generative Adversarial Networks.\n\n")
    f.write("    Args:\n")
    f.write("        num_in_ch (int): Channel number of inputs.\n")
    f.write("        num_out_ch (int): Channel number of outputs.\n")
    f.write("        num_feat (int): Channel number of intermediate features.\n")
    f.write("            Default: 64\n")
    f.write("        num_block (int): Block number in the trunk network. Default: 23\n")
    f.write("        num_grow_ch (int): Channels for growth. Default: 32\n")
    f.write("    \"\"\"\n\n")
    f.write("    def __init__(self, num_in_ch, num_out_ch, num_feat=64, num_block=23, num_grow_ch=32):\n")
    f.write("        super(RRDBNet, self).__init__()\n")
    f.write("        self.conv_first = nn.Conv2d(num_in_ch, num_feat, 3, 1, 1)\n")
    f.write("        self.body = nn.Sequential(*[RRDB(num_feat, num_grow_ch) for _ in range(num_block)])\n")
    f.write("        self.conv_body = nn.Conv2d(num_feat, num_feat, 3, 1, 1)\n")
    f.write("        # upsampling\n")
    f.write("        self.conv_up1 = nn.Conv2d(num_feat, num_feat, 3, 1, 1)\n")
    f.write("        self.conv_up2 = nn.Conv2d(num_feat, num_feat, 3, 1, 1)\n")
    f.write("        self.conv_hr = nn.Conv2d(num_feat, num_feat, 3, 1, 1)\n")
    f.write("        self.conv_last = nn.Conv2d(num_feat, num_out_ch, 3, 1, 1)\n\n")
    f.write("        self.lrelu = nn.LakyReLU(negative_slope=0.2, inplace=True)\n\n")
    f.write("    def forward(self, x):\n")
    f.write("        \"\"\"Forward function.\n\n")
    f.write("        Args:\n")
    f.write("            x (Tensor): Input tensor with shape (n, c, h, w).\n\n")
    f.write("        Returns:\n")
    f.write("            Tensor: Forward results.\n")
    f.write("        \"\"\"\n")
    f.write("        feat = self.conv_first(x)\n")
    f.write("        body_feat = self.conv_body(self.body(feat))\n")
    f.write("        feat = feat + body_feat\n")
    f.write("        # upsampling\n")
    f.write("        feat = self.lrelu(self.conv_up1(F.interpolate(feat, scale_factor=2, mode='nearest')))\n")
    f.write("        feat = self.lrelu(self.conv_up2(F.interpolate(feat, scale_factor=2, mode='nearest')))\n")
    f.write("        out = self.conv_last(self.lrelu(self.conv_hr(feat)))\n")
    f.write("        return out\n")


print(f"Successfully modified and saved {file_path}")

Successfully modified and saved /usr/local/lib/python3.11/dist-packages/basicsr/archs/rrdbnet_arch.py


## 데이터 로딩 및 전처리

### Subtask:
1600x1200 크기의 학습 데이터를 로드하고 모델 학습에 필요한 형태로 전처리하는 코드를 작성합니다.


**Reasoning**:
Import necessary libraries and define a custom PyTorch Dataset for loading and preprocessing image data of size 1600x1200.



In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import os

class CustomSRDataset(Dataset):
    def __init__(self, hr_dir, lr_transform=None, hr_transform=None):
        self.hr_dir = hr_dir
        self.lr_transform = lr_transform
        self.hr_transform = hr_transform
        self.image_filenames = [f for f in os.listdir(hr_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        hr_img_path = os.path.join(self.hr_dir, self.image_filenames[idx])
        hr_img = Image.open(hr_img_path).convert('RGB')

        # Assuming a simple downsampling for creating LR images.
        # In a real scenario, you might load pre-generated LR images
        # or use more sophisticated degradation methods.
        lr_img = hr_img.resize((hr_img.width // 4, hr_img.height // 4), Image.BICUBIC)

        if self.lr_transform:
            lr_img = self.lr_transform(lr_img)
        if self.hr_transform:
            hr_img = self.hr_transform(hr_img)

        return lr_img, hr_img

# Define transformations
hr_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # Normalize to [-1, 1]
])

lr_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # Normalize to [-1, 1]
])

# Example usage (assuming your high-resolution images are in '/content/drive/My Drive/work/hr_images')
# Create a dummy directory and dummy images for demonstration if they don't exist
hr_dummy_dir = '/content/drive/My Drive/work/hr_images'
os.makedirs(hr_dummy_dir, exist_ok=True)
if not os.listdir(hr_dummy_dir):
    print(f"Creating dummy images in {hr_dummy_dir}")
    dummy_img = Image.new('RGB', (1600, 1200), color = 'red')
    dummy_img.save(os.path.join(hr_dummy_dir, 'dummy_image_1.png'))
    dummy_img.save(os.path.join(hr_dummy_dir, 'dummy_image_2.png'))


dataset = CustomSRDataset(hr_dir=hr_dummy_dir,
                          lr_transform=lr_transform,
                          hr_transform=hr_transform)

dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Verify the output shapes
for lr_batch, hr_batch in dataloader:
    print(f"LR Batch Shape: {lr_batch.shape}")
    print(f"HR Batch Shape: {hr_batch.shape}")
    break # Just check one batch

Creating dummy images in /content/drive/My Drive/work/hr_images
LR Batch Shape: torch.Size([2, 3, 300, 400])
HR Batch Shape: torch.Size([2, 3, 1200, 1600])


## 모델 학습 설정

### Subtask:
수정된 모델 아키텍처를 사용하여 학습을 설정합니다. 손실 함수, 옵티마이저, 학습률 스케줄러 등을 정의합니다. 사전 학습된 Real-ESRGAN 모델의 가중치를 로드하여 초기화에 사용할 수 있습니다.


**Reasoning**:
Set up the training configuration by defining the model, loss function, optimizer, and optionally a learning rate scheduler and loading pre-trained weights.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import sys

# Add the directory containing the modified rrdbnet_arch.py to the system path
# This is necessary so that the modified module can be imported correctly
sys.path.insert(0, '/usr/local/lib/python3.11/dist-packages/basicsr/archs')

# Import the modified RRDBNet
from rrdbnet_arch import RRDBNet

# 2. Instantiate the modified RRDBNet model
# Assuming RGB images, so input and output channels are 3.
# num_feat, num_block, and num_grow_ch can be kept as defaults or adjusted.
# The dynamic FC layer size is handled within the modified RRDBNet class.
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 3. Define a loss function
# Using L1 loss as a common choice for super-resolution
criterion = nn.L1Loss()

# 4. Define an optimizer
# Using Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))

# 5. Implement a learning rate scheduler (Optional but recommended)
# Using a StepLR scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.5) # Adjust step_size and gamma as needed

# 6. Load a pre-trained Real-ESRGAN model's state dictionary (Optional)
# You would typically download a pre-trained model file (.pth) and load it.
# Example: Load the 512x512 model weights
# pretrain_model_path = '/content/drive/My Drive/work/Real-ESRGAN_x4plus.pth' # Replace with your path
# try:
#     pretrained_dict = torch.load(pretrain_model_path)
#     model_dict = model.state_dict()

#     # 1. filter out unnecessary keys
#     # The keys for the new FC layer will not be in the pre-trained weights, so they will be filtered out.
#     # The FC layer is initialized dynamically anyway.
#     pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
#     # 2. overwrite entries in the existing state dict
#     model_dict.update(pretrained_dict)
#     # 3. load the new state dict
#     model.load_state_dict(model_dict)
#     print(f"Loaded pre-trained weights from {pretrain_model_path}")
# except FileNotFoundError:
#     print(f"Pre-trained model not found at {pretrain_model_path}. Starting training from scratch.")
# except Exception as e:
#     print(f"Error loading pre-trained model: {e}")

print("Training setup complete: model, criterion, optimizer, and scheduler defined.")

AssertionError: An object named 'RRDBNet' was already registered in 'arch' registry!

**Reasoning**:
The `AssertionError` indicates that `RRDBNet` is already registered in the `ARCH_REGISTRY`. This is likely because the previous modification step re-imported the `basicsr` library, which re-registered the original `RRDBNet`. To fix this, I need to restart the Python kernel to clear the existing registrations and then re-import the modified `rrdbnet_arch.py`.



In [ ]:
# Restart the Python kernel to clear registered modules
# This command is specific to Jupyter environments and will clear the state
# After the kernel restarts, the next code cell will re-execute from a clean state.
import os
os._exit(00)

**Reasoning**:
After restarting the kernel, I need to re-execute the code to set up the training, including importing the modified `RRDBNet` and defining the loss function, optimizer, and scheduler.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import sys
import os

# Add the directory containing the modified rrdbnet_arch.py to the system path
# This is necessary so that the modified module can be imported correctly
# Ensure the path is correct after the kernel restart
sys.path.insert(0, '/usr/local/lib/python3.11/dist-packages/basicsr/archs')

# Import the modified RRDBNet
# This import should now work without the registration error after the kernel restart
from rrdbnet_arch import RRDBNet

# 2. Instantiate the modified RRDBNet model
# Assuming RGB images, so input and output channels are 3.
# num_feat, num_block, and num_grow_ch can be kept as defaults or adjusted.
# The dynamic FC layer size is handled within the modified RRDBNet class.
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 3. Define a loss function
# Using L1 loss as a common choice for super-resolution
criterion = nn.L1Loss()

# 4. Define an optimizer
# Using Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))

# 5. Implement a learning rate scheduler (Optional but recommended)
# Using a StepLR scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.5) # Adjust step_size and gamma as needed

# 6. Load a pre-trained Real-ESRGAN model's state dictionary (Optional)
# You would typically download a pre-trained model file (.pth) and load it.
# Example: Load the 512x512 model weights
# pretrain_model_path = '/content/drive/My Drive/work/Real-ESRGAN_x4plus.pth' # Replace with your path
# try:
#     if os.path.exists(pretrain_model_path):
#         pretrained_dict = torch.load(pretrain_model_path, map_location=device)
#         model_dict = model.state_dict()

#         # 1. filter out unnecessary keys
#         # The keys for the new FC layer will not be in the pre-trained weights, so they will be filtered out.
#         # The FC layer is initialized dynamically anyway.
#         pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
#         # 2. overwrite entries in the existing state dict
#         model_dict.update(pretrained_dict)
#         # 3. load the new state dict
#         model.load_state_dict(model_dict)
#         print(f"Loaded pre-trained weights from {pretrain_model_path}")
#     else:
#          print(f"Pre-trained model not found at {pretrain_model_path}. Starting training from scratch.")

# except Exception as e:
#     print(f"Error loading pre-trained model: {e}")

print("Training setup complete: model, criterion, optimizer, and scheduler defined.")


ModuleNotFoundError: No module named 'torchvision.transforms.functional_tensor'

**Reasoning**:
The `ModuleNotFoundError: No module named 'torchvision.transforms.functional_tensor'` error persists. This indicates a compatibility issue between the installed `basicsr` and `torchvision` versions, where `basicsr` is trying to import a module from `torchvision` that doesn't exist in the current version. Although I attempted to upgrade `torchvision` earlier, it seems it didn't resolve the specific import issue that `basicsr.data.degradations` is facing. Since the goal is to set up the training environment using the *modified* model architecture, and the error is occurring during the import of related `basicsr` data modules (which are not directly needed for *defining* the model, loss, and optimizer in this step), I will try to import `RRDBNet` directly from the modified file path using `importlib.util` to bypass the standard `basicsr` import mechanism that triggers the error.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import sys
import os
import importlib.util

# Define the path to the modified rrdbnet_arch.py
file_path = '/usr/local/lib/python3.11/dist-packages/basicsr/archs/rrdbnet_arch.py'

# Use importlib to load the module directly from the file path
spec = importlib.util.spec_from_file_location("rrdbnet_arch", file_path)
rrdbnet_arch = importlib.util.module_from_spec(spec)
sys.modules["rrdbnet_arch"] = rrdbnet_arch
spec.loader.exec_module(rrdbnet_arch)

# Now, access the RRDBNet class from the loaded module
RRDBNet = rrdbnet_arch.RRDBNet

# 2. Instantiate the modified RRDBNet model
# Assuming RGB images, so input and output channels are 3.
# num_feat, num_block, and num_grow_ch can be kept as defaults or adjusted.
# The dynamic FC layer size is handled within the modified RRDBNet class.
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 3. Define a loss function
# Using L1 loss as a common choice for super-resolution
criterion = nn.L1Loss()

# 4. Define an optimizer
# Using Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))

# 5. Implement a learning rate scheduler (Optional but recommended)
# Using a StepLR scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.5) # Adjust step_size and gamma as needed

# 6. Load a pre-trained Real-ESRGAN model's state dictionary (Optional)
# You would typically download a pre-trained model file (.pth) and load it.
# Example: Load the 512x512 model weights
# pretrain_model_path = '/content/drive/My Drive/work/Real-ESRGAN_x4plus.pth' # Replace with your path
# try:
#     if os.path.exists(pretrain_model_path):
#         pretrained_dict = torch.load(pretrain_model_path, map_location=device)
#         model_dict = model.state_dict()

#         # 1. filter out unnecessary keys
#         # The keys for the new FC layer will not be in the pre-trained weights, so they will be filtered out.
#         # The FC layer is initialized dynamically anyway.
#         pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
#         # 2. overwrite entries in the existing state dict
#         model_dict.update(pretrained_dict)
#         # 3. load the new state dict
#         model.load_state_dict(model_dict)
#         print(f"Loaded pre-trained weights from {pretrain_model_path}")
#     else:
#          print(f"Pre-trained model not found at {pretrain_model_path}. Starting training from scratch.")

# except Exception as e:
#     print(f"Error loading pre-trained model: {e}")

print("Training setup complete: model, criterion, optimizer, and scheduler defined.")

AssertionError: An object named 'RRDBNet' was already registered in 'arch' registry!

**Reasoning**:
The `AssertionError` persists even with `importlib.util`, indicating that the `ARCH_REGISTRY` still holds a reference to `RRDBNet`. Restarting the kernel is the most reliable way to clear these registrations. I will restart the kernel again.



In [ ]:
# Restart the Python kernel to clear registered modules
# This command is specific to Jupyter environments and will clear the state
# After the kernel restarts, the next code cell will re-execute from a clean state.
import os
os._exit(00)

**Reasoning**:
After restarting the kernel, I need to re-execute the code to set up the training, including importing the modified `RRDBNet` and defining the loss function, optimizer, and scheduler. I will use the `importlib.util` method again as it is less likely to trigger the full `basicsr` import chain compared to adding the path to `sys.path`.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import sys
import os
import importlib.util

# Define the path to the modified rrdbnet_arch.py
file_path = '/usr/local/lib/python3.11/dist-packages/basicsr/archs/rrdbnet_arch.py'

# Use importlib to load the module directly from the file path
# Check if the file exists before attempting to load
if not os.path.exists(file_path):
    print(f"Error: Modified model file not found at {file_path}")
else:
    spec = importlib.util.spec_from_file_location("rrdbnet_arch", file_path)
    rrdbnet_arch = importlib.util.module_from_spec(spec)
    sys.modules["rrdbnet_arch"] = rrdbnet_arch
    try:
        spec.loader.exec_module(rrdbnet_arch)

        # Now, access the RRDBNet class from the loaded module
        RRDBNet = rrdbnet_arch.RRDBNet

        # 2. Instantiate the modified RRDBNet model
        # Assuming RGB images, so input and output channels are 3.
        # num_feat, num_block, and num_grow_ch can be kept as defaults or adjusted.
        # The dynamic FC layer size is handled within the modified RRDBNet class.
        model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32)

        # Move model to GPU if available
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        # 3. Define a loss function
        # Using L1 loss as a common choice for super-resolution
        criterion = nn.L1Loss()

        # 4. Define an optimizer
        # Using Adam optimizer
        optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))

        # 5. Implement a learning rate scheduler (Optional but recommended)
        # Using a StepLR scheduler
        scheduler = lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.5) # Adjust step_size and gamma as needed

        # 6. Load a pre-trained Real-ESRGAN model's state dictionary (Optional)
        # You would typically download a pre-trained model file (.pth) and load it.
        # Example: Load the 512x512 model weights
        # pretrain_model_path = '/content/drive/My Drive/work/Real-ESRGAN_x4plus.pth' # Replace with your path
        # try:
        #     if os.path.exists(pretrain_model_path):
        #         pretrained_dict = torch.load(pretrain_model_path, map_location=device)
        #         model_dict = model.state_dict()

        #         # 1. filter out unnecessary keys
        #         # The keys for the new FC layer will not be in the pre-trained weights, so they will be filtered out.
        #         # The FC layer is initialized dynamically anyway.
        #         pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        #         # 2. overwrite entries in the existing state dict
        #         model_dict.update(pretrained_dict)
        #         # 3. load the new state dict
        #         model.load_state_dict(model_dict)
        #         print(f"Loaded pre-trained weights from {pretrain_model_path}")
        #     else:
        #          print(f"Pre-trained model not found at {pretrain_model_path}. Starting training from scratch.")

        # except Exception as e:
        #     print(f"Error loading pre-trained model: {e}")

        print("Training setup complete: model, criterion, optimizer, and scheduler defined.")

    except Exception as e:
        print(f"An error occurred during module execution or model setup: {e}")


An error occurred during module execution or model setup: No module named 'torchvision.transforms.functional_tensor'


**Reasoning**:
The `ModuleNotFoundError: No module named 'torchvision.transforms.functional_tensor'` error is still occurring when executing the modified `rrdbnet_arch.py` module via `importlib`, even after kernel restarts. This strongly suggests an underlying dependency issue within the `basicsr` library itself, where parts of it (specifically the data-related modules, which get imported when `basicsr.archs` is accessed) are incompatible with the installed version of `torchvision`. Since the current subtask is only to set up the training configuration (model, loss, optimizer, scheduler) and not to run the full training loop which would require the data modules, I can try to *temporarily* comment out the problematic import lines within the `rrdbnet_arch.py` file to allow the model definition part to be imported and instantiated. This is a workaround to complete the current subtask, acknowledging that full training might still face issues due to the dependency conflict.



In [ ]:
import os

file_path = '/usr/local/lib/python3.11/dist-packages/basicsr/archs/rrdbnet_arch.py'

try:
    with open(file_path, 'r') as f:
        lines = f.readlines()

    modified_lines = []
    for line in lines:
        # Comment out the problematic import line in the original file
        if "from basicsr.utils.registry import ARCH_REGISTRY" in line:
             modified_lines.append("# Commented out by AI\n")
             modified_lines.append("# " + line)
        else:
             modified_lines.append(line)

    with open(file_path, 'w') as f:
        f.writelines(modified_lines)

    print(f"Successfully commented out problematic import in {file_path}")

except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except Exception as e:
    print(f"An error occurred while modifying the file: {e}")


Successfully commented out problematic import in /usr/local/lib/python3.11/dist-packages/basicsr/archs/rrdbnet_arch.py


**Reasoning**:
I have temporarily commented out the problematic import in `rrdbnet_arch.py`. Now, I will attempt to import the `RRDBNet` class directly from the modified file path using `importlib.util` again. This should allow me to instantiate the model and set up the training configuration without triggering the incompatible imports within the `basicsr` data modules.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import sys
import os
import importlib.util

# Define the path to the modified rrdbnet_arch.py
file_path = '/usr/local/lib/python3.11/dist-packages/basicsr/archs/rrdbnet_arch.py'

# Use importlib to load the module directly from the file path
# Check if the file exists before attempting to load
if not os.path.exists(file_path):
    print(f"Error: Modified model file not found at {file_path}")
else:
    spec = importlib.util.spec_from_file_location("rrdbnet_arch", file_path)
    rrdbnet_arch = importlib.util.module_from_spec(spec)
    sys.modules["rrdbnet_arch"] = rrdbnet_arch
    try:
        spec.loader.exec_module(rrdbnet_arch)

        # Now, access the RRDBNet class from the loaded module
        # Since ARCH_REGISTRY is commented out, we access the class directly
        if hasattr(rrdbnet_arch, 'RRDBNet'):
            RRDBNet = rrdbnet_arch.RRDBNet

            # 2. Instantiate the modified RRDBNet model
            # Assuming RGB images, so input and output channels are 3.
            # num_feat, num_block, and num_grow_ch can be kept as defaults or adjusted.
            # The dynamic FC layer size is handled within the modified RRDBNet class.
            model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32)

            # Move model to GPU if available
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            model.to(device)

            # 3. Define a loss function
            # Using L1 loss as a common choice for super-resolution
            criterion = nn.L1Loss()

            # 4. Define an optimizer
            # Using Adam optimizer
            optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))

            # 5. Implement a learning rate scheduler (Optional but recommended)
            # Using a StepLR scheduler
            scheduler = lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.5) # Adjust step_size and gamma as needed

            # 6. Load a pre-trained Real-ESRGAN model's state dictionary (Optional)
            # You would typically download a pre-trained model file (.pth) and load it.
            # Example: Load the 512x512 model weights
            # pretrain_model_path = '/content/drive/My Drive/work/Real-ESRGAN_x4plus.pth' # Replace with your path
            # try:
            #     if os.path.exists(pretrain_model_path):
            #         pretrained_dict = torch.load(pretrain_model_path, map_location=device)
            #         model_dict = model.state_dict()

            #         # 1. filter out unnecessary keys
            #         # The keys for the new FC layer will not be in the pre-trained weights, so they will be filtered out.
            #         # The FC layer is initialized dynamically anyway.
            #         pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
            #         # 2. overwrite entries in the existing state dict
            #         model_dict.update(pretrained_dict)
            #         # 3. load the new state dict
            #         model.load_state_dict(model_dict)
            #         print(f"Loaded pre-trained weights from {pretrain_model_path}")
            #     else:
            #          print(f"Pre-trained model not found at {pretrain_model_path}. Starting training from scratch.")

            # except Exception as e:
            #     print(f"Error loading pre-trained model: {e}")

            print("Training setup complete: model, criterion, optimizer, and scheduler defined.")
        else:
             print("Error: RRDBNet class not found in the modified module.")


    except Exception as e:
        print(f"An error occurred during module execution or model setup: {e}")


An error occurred during module execution or model setup: name 'ARCH_REGISTRY' is not defined


## 모델 학습

### Subtask:
설정된 환경에서 모델을 학습시키는 코드를 실행합니다. 학습 과정에서 손실 및 성능 지표를 모니터링합니다.


**Reasoning**:
The previous subtask failed due to issues with the `basicsr` library's dependencies and registration mechanism. Before proceeding with training, I need to fix the environment. I will try to address the `ModuleNotFoundError` by downgrading `torchvision` to a version that is likely compatible with the installed `basicsr` library. Then, I will attempt to run the training setup code again, but this time, I will also include the training loop as requested in the current subtask. This combines fixing the environment with the primary goal of this subtask.



In [ ]:
!pip install torchvision==0.11.3

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import sys
import os
import importlib.util
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image

# Define the path to the modified rrdbnet_arch.py
file_path = '/usr/local/lib/python3.11/dist-packages/basicsr/archs/rrdbnet_arch.py'

# Use importlib to load the module directly from the file path
if not os.path.exists(file_path):
    print(f"Error: Modified model file not found at {file_path}")
else:
    spec = importlib.util.spec_from_file_location("rrdbnet_arch", file_path)
    rrdbnet_arch = importlib.util.module_from_spec(spec)
    sys.modules["rrdbnet_arch"] = rrdbnet_arch
    try:
        spec.loader.exec_module(rrdbnet_arch)

        # Access the RRDBNet class from the loaded module
        RRDBNet = rrdbnet_arch.RRDBNet

        # Instantiate the modified RRDBNet model
        model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32)

        # Move model to GPU if available
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        # Define a loss function
        criterion = nn.L1Loss()

        # Define an optimizer
        optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))

        # Implement a learning rate scheduler
        scheduler = lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.5)

        print("Training setup complete: model, criterion, optimizer, and scheduler defined.")

        # Data Loading
        class CustomSRDataset(Dataset):
            def __init__(self, hr_dir, lr_transform=None, hr_transform=None):
                self.hr_dir = hr_dir
                self.lr_transform = lr_transform
                self.hr_transform = hr_transform
                self.image_filenames = [f for f in os.listdir(hr_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

            def __len__(self):
                return len(self.image_filenames)

            def __getitem__(self, idx):
                hr_img_path = os.path.join(self.hr_dir, self.image_filenames[idx])
                hr_img = Image.open(hr_img_path).convert('RGB')
                lr_img = hr_img.resize((hr_img.width // 4, hr_img.height // 4), Image.BICUBIC)

                if self.lr_transform:
                    lr_img = self.lr_transform(lr_img)
                if self.hr_transform:
                    hr_img = self.hr_transform(hr_img)

                return lr_img, hr_img

        hr_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])
        lr_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])
        hr_dummy_dir = '/content/drive/My Drive/work/hr_images'
        dataset = CustomSRDataset(hr_dir=hr_dummy_dir, lr_transform=lr_transform, hr_transform=hr_transform)
        dataloader = DataLoader(dataset, batch_size=1, shuffle=True) # Reduced batch size to 1 to avoid OOM

        # Training Loop
        num_epochs = 5 # Example number of epochs
        for epoch in range(num_epochs):
            for i, (lr_imgs, hr_imgs) in enumerate(dataloader):
                try:
                    lr_imgs = lr_imgs.to(device)
                    hr_imgs = hr_imgs.to(device)

                    optimizer.zero_grad()
                    outputs = model(lr_imgs)
                    loss = criterion(outputs, hr_imgs)
                    loss.backward()
                    optimizer.step()

                    if (i+1) % 1 == 0: # Print every batch
                        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}')
                except Exception as e:
                    print(f"An error occurred during training: {e}")
                    if "out of memory" in str(e):
                        print("CUDA out of memory. Try reducing batch size or model size.")
                        # Optional: clear cache and break
                        torch.cuda.empty_cache()
                        break
                    else:
                        raise e # Re-raise other errors

            scheduler.step()

            # Save model checkpoint
            if (epoch + 1) % 5 == 0:
                torch.save(model.state_dict(), f'model_epoch_{epoch+1}.pth')

    except Exception as e:
        print(f"An error occurred during module execution or model setup: {e}")


ERROR: Ignored the following yanked versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.15.0
ERROR: Could not find a version that satisfies the requirement torchvision==0.11.3 (from versions: 0.15.1, 0.15.2, 0.16.0, 0.16.1, 0.16.2, 0.17.0, 0.17.1, 0.17.2, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 0.20.0, 0.20.1, 0.21.0, 0.22.0, 0.22.1)
ERROR: No matching distribution found for torchvision==0.11.3
An error occurred during module execution or model setup: name 'ARCH_REGISTRY' is not defined


**Reasoning**:
The previous attempt to fix the `ModuleNotFoundError` by installing `torchvision==0.11.3` failed because that version is not available. The `ARCH_REGISTRY` error also persists, likely because the kernel was not fully reset or the import mechanism is still problematic. I will try a different approach to resolve the dependency issue by attempting to install the latest compatible versions of `torch`, `torchvision`, and `torchaudio` explicitly using a command that specifies the CUDA version. This often helps resolve version conflicts. I will also keep the importlib approach to load the modified model file, as directly importing `basicsr` seems to cause issues with the registry.



In [ ]:
# Try installing compatible versions of torch, torchvision, and torchaudio
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import sys
import os
import importlib.util
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image

# Define the path to the modified rrdbnet_arch.py
file_path = '/usr/local/lib/python3.11/dist-packages/basicsr/archs/rrdbnet_arch.py'

# Use importlib to load the module directly from the file path
if not os.path.exists(file_path):
    print(f"Error: Modified model file not found at {file_path}")
else:
    spec = importlib.util.spec_from_file_location("rrdbnet_arch", file_path)
    rrdbnet_arch = importlib.util.module_from_spec(spec)
    sys.modules["rrdbnet_arch"] = rrdbnet_arch
    try:
        spec.loader.exec_module(rrdbnet_arch)

        # Access the RRDBNet class from the loaded module
        # Assuming the ARCH_REGISTRY issue is resolved by a clean import
        if hasattr(rrdbnet_arch, 'RRDBNet'):
             RRDBNet = rrdbnet_arch.RRDBNet
        else:
             print("Error: RRDBNet class not found in the loaded module.")
             raise AttributeError("RRDBNet class not found")


        # Instantiate the modified RRDBNet model
        model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32)

        # Move model to GPU if available
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        # Define a loss function
        criterion = nn.L1Loss()

        # Define an optimizer
        optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))

        # Implement a learning rate scheduler
        scheduler = lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.5)

        print("Training setup complete: model, criterion, optimizer, and scheduler defined.")

        # Data Loading
        class CustomSRDataset(Dataset):
            def __init__(self, hr_dir, lr_transform=None, hr_transform=None):
                self.hr_dir = hr_dir
                self.lr_transform = lr_transform
                self.hr_transform = hr_transform
                self.image_filenames = [f for f in os.listdir(hr_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

            def __len__(self):
                return len(self.image_filenames)

            def __getitem__(self, idx):
                hr_img_path = os.path.join(self.hr_dir, self.image_filenames[idx])
                hr_img = Image.open(hr_img_path).convert('RGB')
                lr_img = hr_img.resize((hr_img.width // 4, hr_img.height // 4), Image.BICUBIC)

                if self.lr_transform:
                    lr_img = self.lr_transform(lr_img)
                if self.hr_transform:
                    hr_img = self.hr_transform(hr_img)

                return lr_img, hr_img

        hr_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])
        lr_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])
        hr_dummy_dir = '/content/drive/My Drive/work/hr_images'
        dataset = CustomSRDataset(hr_dir=hr_dummy_dir, lr_transform=lr_transform, hr_transform=hr_transform)
        dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

        # Training Loop
        num_epochs = 5
        for epoch in range(num_epochs):
            for i, (lr_imgs, hr_imgs) in enumerate(dataloader):
                try:
                    lr_imgs = lr_imgs.to(device)
                    hr_imgs = hr_imgs.to(device)

                    optimizer.zero_grad()
                    outputs = model(lr_imgs)
                    loss = criterion(outputs, hr_imgs)
                    loss.backward()
                    optimizer.step()

                    if (i+1) % 1 == 0:
                        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}')
                except Exception as e:
                    print(f"An error occurred during training: {e}")
                    if "out of memory" in str(e):
                        print("CUDA out of memory. Try reducing batch size or model size.")
                        torch.cuda.empty_cache()
                        break
                    else:
                        raise e

            scheduler.step()

            if (epoch + 1) % 5 == 0:
                torch.save(model.state_dict(), f'model_epoch_{epoch+1}.pth')

    except Exception as e:
        print(f"An error occurred during module execution or model setup: {e}")


Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 117.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 596.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 124.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/1

An error occurred during module execution or model setup: name 'ARCH_REGISTRY' is not defined
